In [16]:
from awpy import Demo
import polars as pl

dem = Demo("mouz-vs-vitality-m3-inferno.dem", verbose=False)
dem.parse(player_props=["health", "armor_value", "yaw", "inventory"])
game_times = dem.parse_ticks(other_props=["game_time", "team_clan_name", 'is_terrorist_timeout', 'is_ct_timeout'])   

In [17]:
from typing import Any, Dict, List

inventory_map = {
    "M9 Bayonet": 1,
    "Butterfly Knife": 2,
    "Karambit": 3,

    
    "USP-S": 4, 
    "P2000": 5,
    "Glock-18": 6,
    "P250": 7, 
    "Dual Berettas": 8,
    "Five-SeveN": 9,
    "Tec-9": 10,
    "Desert Eagle": 11,

        
    "MAC-10": 12,
    "MP9": 13,
    
    
    "AK-47": 14, 
    "Galil AR": 15,
    "M4A1-S": 16,
    "M4A4": 17,
    "FAMAS": 18,
    "AWP": 19,
    "SSG 08": 20,
    
    
    "High Explosive Grenade": 21,
    "Incendiary Grenade": 22,
    "Flashbang": 23,
    "Molotov": 24,
    "Smoke Grenade": 25,
    
    "C4 Explosive": 26,
}

grenade_map = {
    "CFlashbangProjectile": 1,
    "CSmokeGrenadeProjectile": 2,
    "CMolotovProjectile": 3,
    "CHEGrenadeProjectile": 4,
} 

not_weapons = ['knife', 'flashbang', 'smokegrenade']

weapon_map = {
    "M9 Bayonet": 1,
    "Butterfly Knife": 2,
    "Karambit": 3,

    
    "weapon_usp_silencer": 4, 
    "P2000": 5,
    "weapon_glock": 6,
    "weapon_p250": 7, 
    "Dual Berettas": 8,
    "weapon_fiveseven": 9,
    "weapon_tec9": 10,
    "weapon_deagle": 11,

        
    "weapon_mac10": 12,
    "weapon_mp9": 13,
    
    
    "weapon_ak47": 14, 
    "weapon_galilar": 15,
    "weapon_m4a1_silencer": 16,
    "weapon_m4a1": 17,
    "weapon_famas": 18,
    "weapon_awp": 19,
    "weapon_ssg08": 20,
    
    
    "weapon_hegrenade": 21,
    "weapon_incgrenade": 22,
    "Flashbang": 23,
    "weapon_molotov": 24,
    "Smoke Grenade": 25,
    
    "C4 Explosive": 26,
}


def parse_demo_round(dem: Demo, game_times: pl.DataFrame, round_num: int = 1) -> List[Dict[str, Any]]:
    p = dem.ticks['tick', 'X', 'Y', 'side', 'health', 'name', 'yaw', 'inventory']

    p = p.with_columns([
        pl.col('X').cast(pl.Int16),
        pl.col('Y').cast(pl.Int16),
        pl.col('health').cast(pl.UInt8),
        pl.col('yaw').cast(pl.Int16),
        pl.col('inventory').list.eval(pl.element().replace_strict(inventory_map, default=-1)),
        (pl.col('side') == 'ct')
    ])

    grouped_players = p.group_by(pl.col('tick'), maintain_order=True).all()
    
    # grouped_players = grouped_players.with_columns(
    #     pl.col('name').map_elements(
    #         lambda names: [name_to_team.get(name, 'Unknown') for name in names],
    #         return_dtype=pl.List(pl.String)
    #     ).alias('team_clan_names')
    # )
    
    g = dem.grenades['X', 'Y', 'tick', 'grenade_type'].filter(pl.col('Y').is_not_null())

    air_grenades = g.with_columns([
        pl.col('X').cast(pl.Int16),
        pl.col('Y').cast(pl.Int16),
        pl.col('grenade_type').replace_strict(grenade_map, default=-1).cast(pl.Int8),
    ])

    b = dem.events['bomb_planted']['user_X', 'user_Y', 'tick']

    plant = (b.with_columns([
        pl.col('user_X').cast(pl.Int16).alias('X'),
        pl.col('user_Y').cast(pl.Int16).alias('Y'),
    ]).drop(['user_X', 'user_Y']))


    s = dem.events['weapon_fire']['tick', 'user_X', 'user_Y', 'user_yaw', 'weapon'].filter(~pl.col('weapon').str.contains("|".join(not_weapons)))

    shots = (s.with_columns([
        pl.col('user_X').cast(pl.Int16).alias('X'),
        pl.col('user_Y').cast(pl.Int16).alias('Y'),
        pl.col('user_yaw').cast(pl.Int16).alias('yaw'),
        pl.col('weapon').replace_strict(weapon_map, default=-1).cast(pl.Int8),
    ]).drop(['user_X', 'user_Y', 'user_yaw']))


    pd = dem.events['player_death']['tick', 'assistedflash', 'assister_name', 'assister_side', 'attacker_name', 'attacker_side', 'attackerblind', 'attackerinair', 'headshot', 'noscope', 'penetrated', 'thrusmoke', 'weapon', 'user_name', 'user_side']

    kills = pd.with_columns([
        pl.when(pl.col('assister_side') == 'ct')
            .then(1)
            .when(pl.col('assister_side') == 't')
            .then(0)
            .otherwise(-1)
            .cast(pl.Int8)
            .alias('assister_side'),
        pl.when(pl.col('assister_name').is_null())
            .then(pl.lit('N/A'))
            .otherwise(pl.col('assister_name'))
            .alias('assister_name'),
        (pl.col('attacker_side') == 'ct').alias('attacker_side_ct'),
        (pl.col('user_side') == 'ct').alias('user_side_ct'),
        (pl.col('penetrated') == 1).alias('penetrated'),
        pl.col('weapon').replace_strict(weapon_map, default=-1).cast(pl.Int8),
    ])

    # hed = dem.events['hegrenade_detonate']['entityid', 'tick'].with_columns(
    #     pl.col('entityid').cast(pl.Int16)
    # )

    smokes = dem.smokes['entity_id', 'start_tick', 'end_tick', 'thrower_side', 'X', 'Y', 'round_num'].filter(pl.col('round_num') == round_num).drop('round_num')

    smokes = smokes.with_columns([
        pl.col('X').cast(pl.Int16),
        pl.col('Y').cast(pl.Int16),
        pl.col('entity_id').cast(pl.Int16),
        (pl.col('thrower_side') == 'ct')
    ])

    mollies = dem.infernos['entity_id', 'start_tick', 'end_tick', 'thrower_side', 'X', 'Y', 'round_num'].filter(pl.col('round_num') == round_num).drop('round_num')

    mollies = mollies.with_columns([
        pl.col('X').cast(pl.Int16),
        pl.col('Y').cast(pl.Int16),
        pl.col('entity_id').cast(pl.Int16),
        (pl.col('thrower_side') == 'ct')
    ])

    round_info = dem.rounds.filter(pl.col("round_num") == round_num)
    start_tick = round_info[0, "freeze_end"]
    end_tick = round_info[0, "official_end"]
    ticks = list(range(start_tick, end_tick + 1, 16))

    bomb_plant = dem.events['bomb_planted']['tick', 'user_X', 'user_Y'].filter(
        pl.col('tick').is_between(start_tick, end_tick)
    )
    if bomb_plant.height > 0:
        bomb_plant_tick = bomb_plant[0, 'tick']
    else:
        bomb_plant_tick = -1

    tick_list = []

    for tick in ticks:
        try:
            players = grouped_players.row(by_predicate=(pl.col('tick') == tick), named=True)
        except Exception:
            continue

        p = []

        for i in range(len(players["name"])):
            curr = players
            
            p.append({
                "name": curr['name'][i],
                "X": curr['X'][i],
                "Y": curr['Y'][i],
                "side": curr['side'][i],
                "health": curr['health'][i],
                "yaw": curr['yaw'][i],
                "inventory": curr['inventory'][i]
            })
        
        tick_game_time = game_times.filter(pl.col('tick') == tick)[0]
        round_start_game_time = game_times.filter(pl.col('tick') == start_tick)[0]
        
        tick_time = 0
        if bomb_plant_tick != -1 and tick >= bomb_plant_tick:
            # After bomb planted
            bomb_tick_time = game_times.filter(pl.col('tick') == bomb_plant_tick)[0]
            tick_time = 40 - (tick_game_time['game_time'] - bomb_tick_time['game_time'])
        else:
            # Normal round clock
            tick_time = 115 - (tick_game_time['game_time'] - round_start_game_time['game_time'])
            
        tick_time = _format_clock(tick_time[0])
        
        tick_smokes = smokes.filter((tick < pl.col('end_tick')) & (tick > pl.col('start_tick')))
        tick_smokes = tick_smokes['entity_id', 'X', 'Y', 'thrower_side'].to_dicts()

        tick_mollies = mollies.filter((tick < pl.col('end_tick')) & (tick > pl.col('start_tick')))
        tick_mollies = tick_mollies['entity_id', 'X', 'Y', 'thrower_side'].to_dicts()
        
        airborne_grenades = air_grenades.filter(pl.col('tick') == tick).to_dicts()
        
        tick_shots = shots.filter(pl.col('tick').is_between(tick - 8, tick + 8)).to_dicts()

        tick_kills = kills.filter(pl.col('tick').is_between(tick - 8, tick + 8)).to_dicts()
        
        tick_plant = plant.filter(pl.col('tick').is_between(tick - 8, tick + 8)).to_dicts()
        
        tick_list.append({
            "tick": tick,
            "time": 0,
            "players": p,
            "activeSmokes": tick_smokes,
            "activeMolly": tick_mollies,
            "activeGrenades": airborne_grenades,
            "shots": tick_shots,
            "kills": tick_kills,
            "bomb_plant": tick_plant
        })
        
    return tick_list




SECONDARIES = {"USP-S", "Glock-18", "P250", "Five-SeveN", "Dual Berettas", "Tec-9", "Desert Eagle"}
PRIMARIES = {"AK-47", "M4A1-S", "AWP", "FAMAS", "Galil AR"}
GRENADES = {"Flashbang", "Smoke Grenade", "Molotov", "High Explosive Grenade", "Decoy", "Incendiary Grenade"}


def _classify_inventory(items: List[str]) -> Dict[str, Any]:
    knife = items[0] if items else None
    secondary = None
    primary = None
    grenades = []

    for item in items[1:]:
        if item in SECONDARIES:
            secondary = item
        elif item in PRIMARIES:
            primary = item
        elif item in GRENADES:
            grenades.append(item)

    return {
        "knife": knife,
        "secondary": secondary,
        "primary": primary,
        "grenades": grenades
    }

def _format_clock(seconds: float) -> str:
    m = int(seconds) // 60
    s = int(seconds) % 60
    return f"{m}:{s:02}"


def _parse_demo_round(dem: Demo, game_times: pl.DataFrame, round_num: int = 1) -> List[Dict[str, Any]]:
    round_info = dem.rounds.filter(pl.col("round_num") == round_num)
    start_tick = round_info[0, "freeze_end"]
    end_tick = round_info[0, "official_end"]
    tick_list = list(range(start_tick, end_tick + 1, 16))

    bomb_plant = dem.events['bomb_planted']['tick', 'user_X', 'user_Y'].filter(
        pl.col('tick').is_between(start_tick, end_tick)
    )
    if bomb_plant.height > 0:
        bomb_plant_tick = bomb_plant[0, 'tick']
    else:
        bomb_plant_tick = -1

    player_ticks = dem.ticks.filter(
        (pl.col("round_num") == round_num) & (pl.col("tick").is_in(tick_list))
    ).group_by("tick", maintain_order=True).all()

    steamid_to_team = {
        row["steamid"]: row["team_clan_name"]
        for row in game_times.to_dicts()
    }

    grouped_dict = player_ticks.to_dict(as_series=False)
    grouped_dict = {
        tick: {
            "X": grouped_dict["X"][i],
            "Y": grouped_dict["Y"][i],
            "side": grouped_dict["side"][i],
            "health": grouped_dict["health"][i],
            "name": grouped_dict["name"][i],
            "yaw": grouped_dict["yaw"][i],
            "inventory": grouped_dict.get("inventory", [])[i],
            "team_clan_name": [
                steamid_to_team.get(sid) for sid in grouped_dict["steamid"][i]
            ]
        }
        for i, tick in enumerate(grouped_dict["tick"])
    }

    r_smokes = dem.smokes.filter(pl.col("round_num") == round_num).to_pandas()
    r_molly = dem.infernos.filter(pl.col("round_num") == round_num).to_pandas()
    
    he_detonates = dem.events['hegrenade_detonate'].select(['entityid', 'tick']).rename({'tick': 'detonate_tick'})
    he_detonates = he_detonates.unique(subset=["entityid"])
    grenades = dem.grenades.filter(pl.col("X").is_not_null())
    
    grenades_with_detonate = grenades.join(
        he_detonates,
        left_on='entity_id',
        right_on='entityid',
        how='left'
    )
        
    active_grenades = grenades_with_detonate.filter(
        (pl.col("round_num") == round_num) &
        pl.col("X").is_not_null() &
        pl.col("tick").is_in(tick_list) &
        pl.when(pl.col("grenade_type") == "CHEGrenadeProjectile")
        .then(pl.col("tick") < pl.col("detonate_tick"))
        .otherwise(True)
    ).to_pandas()
    
    not_weapons = ['knife', 'flashbang', 'smokegrenade']
    r_shots = dem.events['weapon_fire']['tick', 'user_X', 'user_Y', 'user_yaw', 'weapon'].filter(pl.col('tick') < end_tick).filter(~pl.col('weapon').str.contains("|".join(not_weapons)))
    r_shots = r_shots.with_columns(pl.arange(1, r_shots.height + 1).alias('shot_id'))
    r_shots = r_shots.to_pandas()

    r_kills = dem.events['player_death']['tick', 'assistedflash', 'assister_name', 'assister_side', 'attacker_name', 'attacker_side', 'attackerblind', 'attackerinair', 'headshot', 'noscope', 'penetrated', 'thrusmoke', 'weapon', 'user_name', 'user_side']
    r_kills = r_kills.filter(pl.col('tick').is_between(start_tick, end_tick)).to_pandas()

    grenade_by_tick = active_grenades.groupby("tick")[["thrower", "grenade_type", "X", "Y", "entity_id"]].apply(
        lambda x: x.to_dict("records")
    ).to_dict()

    tick_data_list = []

    for tick in tick_list:

        player_row = grouped_dict.get(tick, {
            "name": [], "X": [], "Y": [], "side": [], "team_clan_name": [], "health": [], "yaw": [], "inventory": []
        })

        players = []
        for i in range(len(player_row["name"])):
            inv = _classify_inventory(player_row["inventory"][i])
            players.append({
                "name": player_row["name"][i],
                "X": player_row["X"][i],
                "Y": player_row["Y"][i],
                "side": player_row["side"][i],
                "health": player_row["health"][i],
                "yaw": player_row["yaw"][i],
                "team_name": player_row['team_clan_name'][i],
                **inv
            })

        active_smokes = r_smokes.query(f"{tick} >= start_tick and {tick} <= end_tick")[["X", "Y", "start_tick", "end_tick", "entity_id"]].to_dict("records")
        active_molly = r_molly.query(f"{tick} >= start_tick and {tick} <= end_tick")[["X", "Y", "start_tick", "end_tick", "entity_id"]].to_dict("records")
        airborne_grenades = grenade_by_tick.get(tick, [])
        
        shots = r_shots.query(f"{tick - 8} <= tick and {tick + 8} >= tick").to_dict("records")

        kills = r_kills[(r_kills['tick'] >= tick - 8) & (r_kills['tick'] <= tick + 8)].to_dict("records")

        for kill in kills:
            # If assister_name is None, replace it with 'N/A' or leave it as None
            kill['assister_name'] = kill.get('assister_name') or 'N/A'
            kill['assister_side'] = kill.get('assister_side') or 'N/A'
            kill['attacker_name'] = kill.get('attacker_name') or 'N/A'
            kill['attacker_side'] = kill.get('attacker_side') or 'N/A'

        tick_game_time = game_times.filter(pl.col('tick') == tick)[0]
        round_start_game_time = game_times.filter(pl.col('tick') == start_tick)[0]

        time = 0
        if bomb_plant_tick != -1 and tick >= bomb_plant_tick:
            # After bomb planted
            bomb_tick_time = game_times.filter(pl.col('tick') == bomb_plant_tick)[0]
            time = 40 - (tick_game_time['game_time'] - bomb_tick_time['game_time'])
        else:
            # Normal round clock
            time = 115 - (tick_game_time['game_time'] - round_start_game_time['game_time'])
            
        time = _format_clock(time[0])

        if bomb_plant.height > 0 and abs(tick - bomb_plant_tick) <= 8:
            first_bomb_plant = bomb_plant.to_pandas().to_dict('records')
        else:
            # Set a default empty dictionary if no bomb plant data
            first_bomb_plant = []

        try:
            tick_data_list.append({
                "tick": tick,
                "time": time,
                "players": players,
                "activeSmokes": active_smokes,
                "activeMolly": active_molly,
                "activeGrenades": airborne_grenades,
                "shots": shots,
                "kills": kills,
                "bomb_plant": first_bomb_plant
            })
        except Exception as e:
            print(f"Error serializing at tick {tick}: {e}")
            raise

    return tick_data_list

In [ ]:


# p = dem.ticks['tick', 'X', 'Y', 'side', 'health', 'name', 'yaw', 'inventory']

# inventory_map = {
#     "M9 Bayonet": 1,
#     "Butterfly Knife": 2,
#     "Karambit": 3,

    
#     "USP-S": 4, 
#     "P2000": 5,
#     "Glock-18": 6,
#     "P250": 7, 
#     "Dual Berettas": 8,
#     "Five-SeveN": 9,
#     "Tec-9": 10,
#     "Desert Eagle": 11,

        
#     "MAC-10": 12,
#     "MP9": 13,
    
    
#     "AK-47": 14, 
#     "Galil AR": 15,
#     "M4A1-S": 16,
#     "M4A4": 17,
#     "FAMAS": 18,
#     "AWP": 19,
#     "SSG 08": 20,
    
    
#     "High Explosive Grenade": 21,
#     "Incendiary Grenade": 22,
#     "Flashbang": 23,
#     "Molotov": 24,
#     "Smoke Grenade": 25,
    
#     "C4 Explosive": 26,
# }

# p = p.with_columns([
#     pl.col('X').cast(pl.Int16),
#     pl.col('Y').cast(pl.Int16),
#     pl.col('health').cast(pl.UInt8),
#     pl.col('yaw').cast(pl.Int16),
#     pl.col('inventory').list.eval(pl.element().replace_strict(inventory_map, default=-1)),
#     (pl.col('side') == 'ct')
# ])

# grouped_players = p.group_by(pl.col('tick'), maintain_order=True).all()

# round_info = dem.rounds.filter(pl.col("round_num") == 4)
# start_tick = round_info[0, "freeze_end"]
# end_tick = round_info[0, "official_end"]
# ticks = list(range(start_tick, end_tick + 1, 16))

# for tick in ticks:
#     players = grouped_players.filter(pl.col('tick') == tick).to_dicts()
            
#     if len(players) == 0:
#         print('tick?', tick)


# grenade_map = {
#     "CFlashbangProjectile": 1,
#     "CSmokeGrenadeProjectile": 2,
#     "CMolotovProjectile": 3,
#     "CHEGrenadeProjectile": 4,
# }

# g = dem.grenades['X', 'Y', 'tick', 'grenade_type'].filter(pl.col('Y').is_not_null())

# g = g.with_columns([
#     pl.col('X').cast(pl.Int16),
#     pl.col('Y').cast(pl.Int16),
#     pl.col('grenade_type').replace_strict(grenade_map, default=-1).cast(pl.Int8),
# ])

# air_grenades = g.group_by('tick').all()

# b = dem.events['bomb_planted']['user_X', 'user_Y', 'tick']

# plant = (b.with_columns([
#     pl.col('user_X').cast(pl.Int16).alias('X'),
#     pl.col('user_Y').cast(pl.Int16).alias('Y'),
# ]).drop(['user_X', 'user_Y']))


# not_weapons = ['knife', 'flashbang', 'smokegrenade']
# weapon_map = {
#     "M9 Bayonet": 1,
#     "Butterfly Knife": 2,
#     "Karambit": 3,

    
#     "weapon_usp_silencer": 4, 
#     "P2000": 5,
#     "weapon_glock": 6,
#     "weapon_p250": 7, 
#     "Dual Berettas": 8,
#     "weapon_fiveseven": 9,
#     "weapon_tec9": 10,
#     "weapon_deagle": 11,

        
#     "weapon_mac10": 12,
#     "weapon_mp9": 13,
    
    
#     "weapon_ak47": 14, 
#     "weapon_galilar": 15,
#     "weapon_m4a1_silencer": 16,
#     "weapon_m4a1": 17,
#     "weapon_famas": 18,
#     "weapon_awp": 19,
#     "weapon_ssg08": 20,
    
    
#     "weapon_hegrenade": 21,
#     "weapon_incgrenade": 22,
#     "Flashbang": 23,
#     "weapon_molotov": 24,
#     "Smoke Grenade": 25,
    
#     "C4 Explosive": 26,
# }
# s = dem.events['weapon_fire']['tick', 'user_X', 'user_Y', 'user_yaw', 'weapon'].filter(~pl.col('weapon').str.contains("|".join(not_weapons)))

# shots = (s.with_columns([
#     pl.col('user_X').cast(pl.Int16).alias('X'),
#     pl.col('user_Y').cast(pl.Int16).alias('Y'),
#     pl.col('user_yaw').cast(pl.Int16).alias('yaw'),
#     pl.col('weapon').replace_strict(weapon_map, default=-1).cast(pl.Int8),
# ]).drop(['user_X', 'user_Y', 'user_yaw']))


# pd = dem.events['player_death']['tick', 'assistedflash', 'assister_name', 'assister_side', 'attacker_name', 'attacker_side', 'attackerblind', 'attackerinair', 'headshot', 'noscope', 'penetrated', 'thrusmoke', 'weapon', 'user_name', 'user_side']

# kills = pd.with_columns([
#     pl.when(pl.col('assister_side') == 'ct')
#       .then(1)
#       .when(pl.col('assister_side') == 't')
#       .then(0)
#       .otherwise(-1)
#       .cast(pl.Int8)
#       .alias('assister_side'),
#     (pl.col('attacker_side') == 'ct').alias('attacker_side_ct'),
#     (pl.col('user_side') == 'ct').alias('user_side_ct'),
#     (pl.col('penetrated') == 1).alias('penetrated'),
#     pl.col('weapon').replace_strict(weapon_map, default=-1).cast(pl.Int8),
# ])

# # hed = dem.events['hegrenade_detonate']['entityid', 'tick'].with_columns(
# #     pl.col('entityid').cast(pl.Int16)
# # )


# smokes = dem.smokes['entity_id', 'start_tick', 'end_tick', 'thrower_side', 'X', 'Y', 'round_num'].filter(pl.col('round_num') == 4).drop('round_num')

# smokes = smokes.with_columns([
#     pl.col('X').cast(pl.Int16),
#     pl.col('Y').cast(pl.Int16),
#     pl.col('entity_id').cast(pl.Int16),
#     (pl.col('thrower_side') == 'ct')
# ])

# mollies = dem.infernos['entity_id', 'start_tick', 'end_tick', 'thrower_side', 'X', 'Y', 'round_num'].filter(pl.col('round_num') == 4).drop('round_num')

# mollies = mollies.with_columns([
#     pl.col('X').cast(pl.Int16),
#     pl.col('Y').cast(pl.Int16),
#     pl.col('entity_id').cast(pl.Int16),
#     (pl.col('thrower_side') == 'ct')
# ])

# # game_times['is_terrorist_timeout', 'is_ct_timeout', 'team_clan_name', 'game_time', 'tick', 'name']



# round_info = dem.rounds.filter(pl.col("round_num") == 4)
# start_tick = round_info[0, "freeze_end"]
# end_tick = round_info[0, "official_end"]
# ticks = list(range(start_tick, end_tick + 1, 16))

# tick_list = []

# from polars.exceptions import NoRowsReturnedError
# for tick in ticks:
    
#     players = grouped_players.row(by_predicate=(pl.col('tick') == tick), named=True)
    
#     tick_time = 0
#     # if bomb_plant_tick != -1 and tick >= bomb_plant_tick:
#     #     # After bomb planted
#     #     bomb_tick_time = game_times.filter(pl.col('tick') == bomb_plant_tick)[0]
#     #     tick_time = 40 - (tick_game_time['game_time'] - bomb_tick_time['game_time'])
#     # else:
#     #     # Normal round clock
#     #     titick_timeme = 115 - (tick_game_time['game_time'] - round_start_game_time['game_time'])
        
#     # tick_time = _format_clock(time[0])
    
#     tick_smokes = smokes.filter((tick < pl.col('end_tick')) & (tick > pl.col('start_tick')))
#     tick_smokes = tick_smokes['entity_id', 'X', 'Y', 'thrower_side'].to_dicts()
#     tick_mollies = mollies.filter((tick < pl.col('end_tick')) & (tick > pl.col('start_tick')))
#     tick_mollies = tick_mollies['entity_id', 'X', 'Y', 'thrower_side'].to_dicts()
    
#     try:
#         airborne_grenades = air_grenades.row(by_predicate=(pl.col('tick') == tick), named=True)
#     except NoRowsReturnedError:
#         airborne_grenades = []
    
#     tick_shots = shots.filter(pl.col('tick').is_between(tick - 8, tick + 8)).to_dicts()
    
#     tick_kills = kills.filter(pl.col('tick').is_between(tick - 8, tick + 8)).to_dicts()
    
#     tick_plant = plant.filter(pl.col('tick').is_between(tick - 8, tick + 8)).to_dicts()
    
#     tick_list.append({
#         "tick": tick,
#         "time": tick_time,
#         "players": players,
#         "activeSmokes": tick_smokes,
#         "activeMolly": tick_mollies,
#         "activeGrenades": airborne_grenades,
#         "shots": tick_shots,
#         "kills": tick_kills,
#         "bomb_plant": tick_plant
#     })
    
# print(tick_list[100])

old version takes 2.997 seconds
new version takes 2.608 seconds


In [252]:
print(parse_demo_round(dem, game_times))

[{'tick': 22331, 'time': 0, 'players': {'tick': 22331, 'X': [2456, -1520, -1662, 2493, -1586, -1657, 2353, -1675, 2292, 2397], 'Y': [2153, 430, 288, 2090, 440, 419, 1977, 351, 2027, 2079], 'side': [True, False, False, True, False, False, True, False, True, True], 'health': [100, 100, 100, 100, 100, 100, 100, 100, 100, 100], 'name': ['Spinx', 'ropz', 'ZywOo', 'Brollan', 'apEX', 'mezii', 'torzsi', 'flameZ', 'Jimpphat', 'xertioN'], 'yaw': [-110, -74, -12, 137, -44, -13, -90, -70, -69, 138], 'inventory': [[1, 4], [3, 6, 14, 24, 21, 25, 23], [3, 6, 24, 25, 21, 16, 26, 23], [3, 25, 15, 4], [1, 11, 21, 24, 13, 23, 25], [1, 25, 23, 9, 24, 21, 14], [2, 9], [3, 10, 12, 25, 23, 24], [1, 4], [2, 4]]}, 'activeSmokes': [], 'activeMolly': [], 'activeGrenades': [], 'shots': [], 'kills': [], 'bomb_plant': []}, {'tick': 22347, 'time': 0, 'players': {'tick': 22347, 'X': [2435, -1512, -1634, 2491, -1556, -1637, 2352, -1665, 2300, 2374], 'Y': [2135, 402, 278, 2119, 440, 400, 1947, 324, 2001, 2098], 'side':

In [41]:
def parse_demo_round(dem: Demo, game_times: pl.DataFrame, round_num: int = 1) -> List[Dict[str, Any]]:
    p = dem.ticks['tick', 'X', 'Y', 'side', 'health', 'name', 'yaw', 'inventory']

    inventory_map = {
        "M9 Bayonet": 1,
        "Butterfly Knife": 2,
        "Karambit": 3,

        
        "USP-S": 4, 
        "P2000": 5,
        "Glock-18": 6,
        "P250": 7, 
        "Dual Berettas": 8,
        "Five-SeveN": 9,
        "Tec-9": 10,
        "Desert Eagle": 11,

            
        "MAC-10": 12,
        "MP9": 13,
        
        
        "AK-47": 14, 
        "Galil AR": 15,
        "M4A1-S": 16,
        "M4A4": 17,
        "FAMAS": 18,
        "AWP": 19,
        "SSG 08": 20,
        
        
        "High Explosive Grenade": 21,
        "Incendiary Grenade": 22,
        "Flashbang": 23,
        "Molotov": 24,
        "Smoke Grenade": 25,
        
        "C4 Explosive": 26,
    }

    p = p.with_columns([
        pl.col('X').cast(pl.Int16),
        pl.col('Y').cast(pl.Int16),
        pl.col('health').cast(pl.UInt8),
        pl.col('yaw').cast(pl.Int16),
        pl.col('inventory').list.eval(pl.element().replace_strict(inventory_map, default=-1)),
        (pl.col('side') == 'ct')
    ])

    grouped_players = p.group_by(pl.col('tick'), maintain_order=True).all()


    grenade_map = {
        "CFlashbangProjectile": 1,
        "CSmokeGrenadeProjectile": 2,
        "CMolotovProjectile": 3,
        "CHEGrenadeProjectile": 4,
    }

    g = dem.grenades['X', 'Y', 'tick', 'grenade_type'].filter(pl.col('Y').is_not_null())

    air_grenades = g.with_columns([
        pl.col('X').cast(pl.Int16),
        pl.col('Y').cast(pl.Int16),
        pl.col('grenade_type').replace_strict(grenade_map, default=-1).cast(pl.Int8),
    ])

    # air_grenades = g.group_by('tick').all()

    b = dem.events['bomb_planted']['user_X', 'user_Y', 'tick']

    plant = (b.with_columns([
        pl.col('user_X').cast(pl.Int16).alias('X'),
        pl.col('user_Y').cast(pl.Int16).alias('Y'),
    ]).drop(['user_X', 'user_Y']))


    not_weapons = ['knife', 'flashbang', 'smokegrenade']
    weapon_map = {
        "M9 Bayonet": 1,
        "Butterfly Knife": 2,
        "Karambit": 3,

        
        "weapon_usp_silencer": 4, 
        "P2000": 5,
        "weapon_glock": 6,
        "weapon_p250": 7, 
        "Dual Berettas": 8,
        "weapon_fiveseven": 9,
        "weapon_tec9": 10,
        "weapon_deagle": 11,

            
        "weapon_mac10": 12,
        "weapon_mp9": 13,
        
        
        "weapon_ak47": 14, 
        "weapon_galilar": 15,
        "weapon_m4a1_silencer": 16,
        "weapon_m4a1": 17,
        "weapon_famas": 18,
        "weapon_awp": 19,
        "weapon_ssg08": 20,
        
        
        "weapon_hegrenade": 21,
        "weapon_incgrenade": 22,
        "Flashbang": 23,
        "weapon_molotov": 24,
        "Smoke Grenade": 25,
        
        "C4 Explosive": 26,
    }
    s = dem.events['weapon_fire']['tick', 'user_X', 'user_Y', 'user_yaw', 'weapon'].filter(~pl.col('weapon').str.contains("|".join(not_weapons)))

    shots = (s.with_columns([
        pl.col('user_X').cast(pl.Int16).alias('X'),
        pl.col('user_Y').cast(pl.Int16).alias('Y'),
        pl.col('user_yaw').cast(pl.Int16).alias('yaw'),
        pl.col('weapon').replace_strict(weapon_map, default=-1).cast(pl.Int8),
    ]).drop(['user_X', 'user_Y', 'user_yaw']))


    pd = dem.events['player_death']['tick', 'assistedflash', 'assister_name', 'assister_side', 'attacker_name', 'attacker_side', 'attackerblind', 'attackerinair', 'headshot', 'noscope', 'penetrated', 'thrusmoke', 'weapon', 'user_name', 'user_side']

    kills = pd.with_columns([
        pl.when(pl.col('assister_side') == 'ct')
            .then(1)
            .when(pl.col('assister_side') == 't')
            .then(0)
            .otherwise(-1)
            .cast(pl.Int8)
            .alias('assister_side'),
        pl.when(pl.col('assister_name').is_null())
            .then(pl.lit('N/A'))
            .otherwise(pl.col('assister_name'))
            .alias('assister_name'),
        (pl.col('attacker_side') == 'ct').alias('attacker_side_ct'),
        (pl.col('user_side') == 'ct').alias('user_side_ct'),
        (pl.col('penetrated') == 1).alias('penetrated'),
        pl.col('weapon').replace_strict(weapon_map, default=-1).cast(pl.Int8),
    ])

    # hed = dem.events['hegrenade_detonate']['entityid', 'tick'].with_columns(
    #     pl.col('entityid').cast(pl.Int16)
    # )


    smokes = dem.smokes['entity_id', 'start_tick', 'end_tick', 'thrower_side', 'X', 'Y', 'round_num'].filter(pl.col('round_num') == round_num).drop('round_num')

    smokes = smokes.with_columns([
        pl.col('X').cast(pl.Int16),
        pl.col('Y').cast(pl.Int16),
        pl.col('entity_id').cast(pl.Int16),
        (pl.col('thrower_side') == 'ct')
    ])

    mollies = dem.infernos['entity_id', 'start_tick', 'end_tick', 'thrower_side', 'X', 'Y', 'round_num'].filter(pl.col('round_num') == round_num).drop('round_num')

    mollies = mollies.with_columns([
        pl.col('X').cast(pl.Int16),
        pl.col('Y').cast(pl.Int16),
        pl.col('entity_id').cast(pl.Int16),
        (pl.col('thrower_side') == 'ct')
    ])

    # game_times['is_terrorist_timeout', 'is_ct_timeout', 'team_clan_name', 'game_time', 'tick', 'name']



    round_info = dem.rounds.filter(pl.col("round_num") == round_num)
    start_tick = round_info[0, "freeze_end"]
    end_tick = round_info[0, "official_end"]
    ticks = list(range(start_tick, end_tick + 1, 16))

    tick_list = []

    for tick in ticks:
        
        players = grouped_players.row(by_predicate=(pl.col('tick') == tick), named=True)

        p = []
        try:
            for i in range(len(players["name"])):
                curr = players
            
                p.append({
                    "name": curr['name'][i],
                    "X": curr['X'][i],
                    "Y": curr['Y'][i],
                    "side": curr['side'][i],
                    "health": curr['health'][i],
                    "yaw": curr['yaw'][i],
                    "team_name": "",  # If you have team info elsewhere, replace
                    "inventory": curr['inventory'][i]
                })
        except Exception:
            print('here?', players)
            
        
        tick_time = 0
        # if bomb_plant_tick != -1 and tick >= bomb_plant_tick:
        #     # After bomb planted
        #     bomb_tick_time = game_times.filter(pl.col('tick') == bomb_plant_tick)[0]
        #     tick_time = 40 - (tick_game_time['game_time'] - bomb_tick_time['game_time'])
        # else:
        #     # Normal round clock
        #     titick_timeme = 115 - (tick_game_time['game_time'] - round_start_game_time['game_time'])
            
        # tick_time = _format_clock(time[0])
        
        tick_smokes = smokes.filter((tick < pl.col('end_tick')) & (tick > pl.col('start_tick')))
        tick_smokes = tick_smokes['entity_id', 'X', 'Y', 'thrower_side'].to_dicts()

        tick_mollies = mollies.filter((tick < pl.col('end_tick')) & (tick > pl.col('start_tick')))
        tick_mollies = tick_mollies['entity_id', 'X', 'Y', 'thrower_side'].to_dicts()
        
        airborne_grenades = air_grenades.filter(pl.col('tick') == tick).to_dicts()
        
        tick_shots = shots.filter(pl.col('tick').is_between(tick - 8, tick + 8)).to_dicts()

        tick_kills = kills.filter(pl.col('tick').is_between(tick - 8, tick + 8)).to_dicts()
        
        tick_plant = plant.filter(pl.col('tick').is_between(tick - 8, tick + 8)).to_dicts()
        
        tick_list.append({
            "tick": tick,
            "time": tick_time,
            "players": p,
            "activeSmokes": tick_smokes,
            "activeMolly": tick_mollies,
            "activeGrenades": airborne_grenades,
            "shots": tick_shots,
            "kills": tick_kills,
            "bomb_plant": tick_plant
        })
        
    return tick_list

In [30]:
p = dem.ticks['tick', 'X', 'Y', 'side', 'health', 'name', 'yaw', 'inventory']

p = p.with_columns([
    pl.col('X').cast(pl.Int16),
    pl.col('Y').cast(pl.Int16),
    pl.col('health').cast(pl.UInt8),
    pl.col('yaw').cast(pl.Int16),
    pl.col('inventory').list.eval(pl.element().replace_strict(inventory_map, default=-1)),
    (pl.col('side') == 'ct')
])

grouped_players = p.group_by(pl.col('tick'), maintain_order=True).all()

name_to_team = {
    row["name"]: row["team_clan_name"]
    for row in game_times.to_dicts()
}

print(name_to_team)


grouped_players = grouped_players.with_columns(
    pl.col('name').map_elements(
        lambda names: [name_to_team.get(name, 'Unknown') for name in names],
        return_dtype=pl.List(pl.String)
    ).alias('team_clan_names')
)
print(grouped_players['name', 'team_clan_names'])


{'Spinx': 'MOUZ', 'ropz': 'Team Vitality', 'ZywOo': 'Team Vitality', 'Brollan': 'MOUZ', 'apEX': 'Team Vitality', 'mezii': 'Team Vitality', 'torzsi': 'MOUZ', 'flameZ': 'Team Vitality', 'Jimpphat': 'MOUZ', 'xertioN': 'MOUZ'}
shape: (138_319, 2)
┌────────────────────────────────┬─────────────────────────────────┐
│ name                           ┆ team_clan_names                 │
│ ---                            ┆ ---                             │
│ list[str]                      ┆ list[str]                       │
╞════════════════════════════════╪═════════════════════════════════╡
│ ["Spinx", "ropz", … "xertioN"] ┆ ["MOUZ", "Team Vitality", … "M… │
│ ["Spinx", "ropz", … "xertioN"] ┆ ["MOUZ", "Team Vitality", … "M… │
│ ["Spinx", "ropz", … "xertioN"] ┆ ["MOUZ", "Team Vitality", … "M… │
│ ["Spinx", "ropz", … "xertioN"] ┆ ["MOUZ", "Team Vitality", … "M… │
│ ["Spinx", "ropz", … "xertioN"] ┆ ["MOUZ", "Team Vitality", … "M… │
│ …                              ┆ …                               

In [18]:
import time


def run_test():
    start = time.perf_counter()
    tick_list = _parse_demo_round(dem, game_times, 3)
    end = time.perf_counter()
    print(f'old version takes {end - start:.3f} seconds')
    
    start = time.perf_counter()
    tick_list2 = parse_demo_round(dem, game_times, 3)
    end = time.perf_counter()
    print(f'new version takes {end - start:.3f} seconds')

    # try:
    #     assert tick_list == tick_list2
    #     print('✅ Test passed: outputs match!')
    # except AssertionError:
    #     print('❌ Test failed: outputs do not match!')
    #     raise  # re-raise to still see the traceback if you want
    
run_test()

# r = game_times.row(by_predicate=(pl.col('name') == 'Spinx'))
# print(r)

# Perform a join between 'p' and 'game_times' on the 'name' column
# p = p.join(game_times, on='name', how='left')

# print(p)


old version takes 2.953 seconds
new version takes 0.833 seconds
